In [1]:
import pandas as pd
import numpy as np
import pickle as pk

In [4]:
file_name = '1_min'
df = pd.read_csv(file_name + '.csv')
df['behavior'] = np.zeros(len(df)).astype(np.int)

In [5]:
intention_2_action_delay = 3000

acc_threshold = 1

# 0 for changing to left
# 1 for changing to right
# 2 for following

next_lane_change_time = dict()
next_lane_change_direct = dict()
next_vel_change_time = dict()
next_vel_change_direct = dict()

def classify_behavior(v_id):
    if next_lane_change_time[v_id] > -1 and next_lane_change_time[v_id] - r.Global_Time < intention_2_action_delay:
        return next_lane_change_direct[v_id]
    return 2
#     if next_vel_change_time[v_id] > -1 and next_vel_change_time[v_id] - r.Global_Time < intention_2_action_delay:
#         return next_vel_change_direct[v_id] + 2
#     return 4

lane_id = dict()
behavior_seq = dict()
cnt = np.zeros((5, 5))
for i in reversed(range(len(df))):
    
    r = df.iloc[i]
    v_id = r.Vehicle_ID
    
    if v_id not in lane_id.keys():
        lane_id[v_id] = r.Lane_ID
        next_lane_change_time[v_id] = -1
        next_vel_change_time[v_id] = -1
        behavior_seq[v_id] = list()
    
    if r.Lane_ID != lane_id[v_id]:
        next_lane_change_time[v_id] = r.Global_Time
        next_lane_change_direct[v_id] = int(r.Lane_ID > lane_id[v_id])
        lane_id[v_id] = r.Lane_ID
    
#     if abs(r.v_Acc) > acc_threshold:
#         next_vel_change_time[v_id] = r.Global_Time
#         next_vel_change_direct[v_id] = int(r.v_Acc > 0)
    
    bhv = classify_behavior(v_id)
    behavior_seq[v_id].append(bhv)
    df.at[i,'behavior']= bhv
    

In [6]:
for v_id in behavior_seq.keys():
    for i in range(len(behavior_seq[v_id])-1):
        cnt[behavior_seq[v_id][i+1]][behavior_seq[v_id][i]] += 1 # because we append by reverse order.

A = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        A[i,j] = cnt[i,j] / sum(cnt[i,:])

print(A)

[[  9.66057441e-01   0.00000000e+00   3.39425587e-02]
 [  5.42888165e-04   9.66340934e-01   3.31161781e-02]
 [  2.33164036e-04   6.08254007e-04   9.99158582e-01]]


In [8]:
f = open('A.txt','wb')
pk.dump(A, f)

In [9]:
A

array([[  9.66057441e-01,   0.00000000e+00,   3.39425587e-02],
       [  5.42888165e-04,   9.66340934e-01,   3.31161781e-02],
       [  2.33164036e-04,   6.08254007e-04,   9.99158582e-01]])

In [12]:
f = open('A.txt', 'rb')
AA = pk.load(f)

In [13]:
AA

array([[  9.66057441e-01,   0.00000000e+00,   3.39425587e-02],
       [  5.42888165e-04,   9.66340934e-01,   3.31161781e-02],
       [  2.33164036e-04,   6.08254007e-04,   9.99158582e-01]])